In [13]:
output =  yf.Ticker('AAPL')
print(output.financials)

                                                        2023-09-30  \
Tax Effect Of Unusual Items                                    0.0   
Tax Rate For Calcs                                           0.147   
Normalized EBITDA                                   129188000000.0   
Net Income From Continuing Operation Net Minori...   96995000000.0   
Reconciled Depreciation                              11519000000.0   
Reconciled Cost Of Revenue                          214137000000.0   
EBITDA                                              129188000000.0   
EBIT                                                117669000000.0   
Net Interest Income                                   -183000000.0   
Interest Expense                                      3933000000.0   
Interest Income                                       3750000000.0   
Normalized Income                                    96995000000.0   
Net Income From Continuing And Discontinued Ope...   96995000000.0   
Total Expenses      

In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class StockAnalyzer:
    def __init__(self, ticker_symbol):
        self.ticker = yf.Ticker(ticker_symbol)
        self.symbol = ticker_symbol
        
    def get_financial_metrics(self):
        """Calculate key financial metrics"""
        try:
            # Get financial statements
            income_stmt = self.ticker.financials
            balance_sheet = self.ticker.balance_sheet
            cashflow = self.ticker.cashflow
            
            metrics = {}
            
            # Growth Metrics
            metrics['revenue_growth'] = self._calculate_growth_rate(income_stmt.loc['Total Revenue'])
            metrics['eps_growth'] = self._calculate_growth_rate(self.ticker.history(period="5y")['Close'])
            
            # Profitability Metrics
            metrics['gross_margin'] = (income_stmt.loc['Gross Profit'] / 
                                     income_stmt.loc['Total Revenue']).iloc[0]
            metrics['operating_margin'] = (income_stmt.loc['Operating Income'] / 
                                         income_stmt.loc['Total Revenue']).iloc[0]
            metrics['net_margin'] = (income_stmt.loc['Net Income'] / 
                                   income_stmt.loc['Total Revenue']).iloc[0]
            
            # Balance Sheet Metrics
            if 'Total Liab' in balance_sheet.index and 'Total Stockholder Equity' in balance_sheet.index:
                metrics['debt_to_equity'] = (balance_sheet.loc['Total Liab'] / 
                                           balance_sheet.loc['Total Stockholder Equity']).iloc[0]
            else:
                print(f"Warning: 'Total Liab' or 'Total Stockholder Equity' not found for {self.symbol}")
                metrics['debt_to_equity'] = None
            
            if 'Total Current Assets' in balance_sheet.index and 'Total Current Liabilities' in balance_sheet.index:
                metrics['current_ratio'] = (balance_sheet.loc['Total Current Assets'] / 
                                          balance_sheet.loc['Total Current Liabilities']).iloc[0]
            else:
                print(f"Warning: 'Total Current Assets' or 'Total Current Liabilities' not found for {self.symbol}")
                metrics['current_ratio'] = None
            
            # Cash Flow Metrics
            metrics['fcf_yield'] = (cashflow.loc['Free Cash Flow'] / 
                                  (self.ticker.info['marketCap'])).iloc[0]
            
            # Valuation Metrics
            metrics['pe_ratio'] = self.ticker.info.get('trailingPE', None)
            metrics['pb_ratio'] = self.ticker.info.get('priceToBook', None)
            metrics['ps_ratio'] = self.ticker.info.get('priceToSalesTrailing12Months', None)
            return metrics
        
        except Exception as e:
            print(f"Error calculating financial metrics for {self.symbol}: {str(e)}")
            return None
            
            
    def get_technical_indicators(self, period='1y'):
        """Calculate technical indicators"""
        hist = self.ticker.history(period=period)
        
        indicators = {}
        
        # Moving averages
        indicators['SMA_50'] = hist['Close'].rolling(window=50).mean().iloc[-1]
        indicators['SMA_200'] = hist['Close'].rolling(window=200).mean().iloc[-1]
        
        # RSI
        delta = hist['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        indicators['RSI'] = (100 - (100 / (1 + rs))).iloc[-1]
        
        # Volume analysis
        indicators['volume_avg'] = hist['Volume'].mean()
        indicators['volume_recent'] = hist['Volume'].iloc[-1]
        
        return indicators
    
    def get_risk_metrics(self):
        """Calculate risk metrics"""
        hist = self.ticker.history(period='1y')
        
        risk_metrics = {}
        
        # Volatility
        risk_metrics['volatility'] = hist['Close'].pct_change().std() * np.sqrt(252)
        
        # Beta
        market = yf.download('^GSPC', start=hist.index[0].strftime('%Y-%m-%d'), end=hist.index[-1].strftime('%Y-%m-%d'))
        returns = hist['Close'].pct_change()[1:]
        market_returns = market['Close'].pct_change()[1:]
        covariance = returns.cov(market_returns)
        market_variance = market_returns.var()
        risk_metrics['beta'] = covariance / market_variance
        
        return risk_metrics
    
    def _calculate_growth_rate(self, series):
        """Calculate compound annual growth rate"""
        first_value = series.iloc[-1]
        last_value = series.iloc[0]
        num_periods = len(series) - 1
        if first_value > 0 and last_value > 0:
            return (last_value / first_value) ** (1 / num_periods) - 1
        return None
    
    def generate_report(self):
        """Generate comprehensive analysis report"""
        metrics = self.get_financial_metrics()
        technical = self.get_technical_indicators()
        risk = self.get_risk_metrics()
        
        report = {
            'symbol': self.symbol,
            'analysis_date': datetime.now().strftime('%Y-%m-%d'),
            'metrics': metrics,
            'technical': technical,
            'risk': risk,
            'score': self._calculate_score(metrics, technical, risk)
        }
        
        return report
    
    def _calculate_score(self, metrics, technical, risk):
        """Calculate overall stock score based on various factors"""
        score = 0
        
        # Example scoring logic (customize based on your preferences)
        if metrics['pe_ratio'] and metrics['pe_ratio'] < 20:
            score += 10
        if metrics['debt_to_equity'] < 1:
            score += 10
        if metrics['current_ratio'] > 1.5:
            score += 10
        if risk['beta'] < 1:
            score += 10
        if technical['RSI'] < 70:  # Not overbought
            score += 10
            
        return score

# Example usage
def analyze_stocks(ticker_list):
    results = []
    for ticker in ticker_list:
        try:
            analyzer = StockAnalyzer(ticker)
            report = analyzer.generate_report()
            results.append(report)
        except Exception as e:
            print(f"Error analyzing {ticker}: {str(e)}")
    
    return pd.DataFrame(results)

In [8]:
# List of stock tickers
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 
          'TSLA', 'LLY', 'V', 'TSM', 'JNJ', 'JPM', 'NVO', 
          'UNH', 'PG', 'HD', 'IDCBY', 'LVMUY', 'MA', 'DIS', 
          'PYPL', 'ADBE']

# Analyze stocks and save to CSV
df = analyze_stocks(stocks)
df.to_csv('stocks_analysis.csv', index=False)

[*********************100%%**********************]  1 of 1 completed
Error analyzing AAPL: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing MSFT: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing GOOGL: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing AMZN: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing NVDA: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing META: Cannot join tz-naive with tz-aware DatetimeIndex
[*********************100%%**********************]  1 of 1 completed
Error analyzing TSLA: Cannot join tz-naive with tz-aware DatetimeIndex
[******************

KeyboardInterrupt: 